In [1]:
import ipanema
import numpy as np
ipanema.initialize('cuda',1)
from scipy import special
import matplotlib.pyplot as plt
%matplotlib widget
import ipywidgets as widgets
import uproot3 as uproot

Welcome to JupyROOT 6.22/02


In [2]:
cut1 = "Bu_IPCHI2_OWNPV<9 & Jpsi_M>3075 & Jpsi_M<3120"
#cut1 = f"{cut1} & Bu_M>5100 & Bu_M<5450"
cut2 = "Jpsi_ENDVERTEX_CHI2<9 & Bu_ENDVERTEX_CHI2<27 & log10(Bu_TAU)>-3.1"
cut2 = f"{cut1} & {cut2}"
rd = uproot.open("/scratch46/marcos.romero/Bu2JpsiKplus5r.root")
rd = rd['DecayTree'].pandas.df().query(cut2)
#rd = ipanema.Sample.from_pandas(rd)
mass = rd['Bu_M']

In [2]:
prog = THREAD.compile(
"""
#define USE_DOUBLE 1
#include <ipanema/core.c>
#include <ipanema/complex.c>
#include <ipanema/special.c>
#include <ipanema/lineshapes.c>
#include <exposed/kernels.ocl>
""", compiler_options=[f"-I{ipanema.IPANEMALIB}"])

x = np.linspace(4900,5500,200).astype(np.float64)
xd = ipanema.ristra.allocate(x)
yd = ipanema.ristra.allocate(0*x)

def shoulder(x, m0=5100, c=5, p=20):

  prog.py_physbkg(yd, xd, np.float64(m0), np.float64(c), np.float64(p),
                global_size=(len(x)))
  return ipanema.ristra.get(yd)

In [5]:
@widgets.interact(m0=(5000, 5300,5), c=(-10, 10, 0.1), p=(-10, 10, 0.1))
def show_ipatia(m0, c, p):
  fig, ax = ipanema.plotting.axes_plot()  
  #ip = shoulder(x, 5100, 100, -10)
  #y = ip/np.trapz(ip,x)
  #ax.plot(x, y)  
  ip = shoulder(x, m0, c, p)
  y = ip/np.trapz(ip,x)
  ax.plot(x, y)
  #ax.legend()

interactive(children=(IntSlider(value=5150, description='m0', max=5300, min=5000, step=5), FloatSlider(value=0…